%run ../ipynb_util_tars.py

In [58]:
import os

from qdrant_client import QdrantClient, models

client = QdrantClient(
    os.getenv("QDRANT_API_URL"),
    port=os.getenv("QDRANT_API_PORT"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

In [59]:
# scroll over points that HAVE labels ("labels" field is not null)
scroll_result = client.scroll(
    collection_name="publications",
    limit=500,
    with_payload=["id", "labels", "xai", "description"],
    scroll_filter=models.Filter(
        must_not=[
            models.IsNullCondition(
                is_null=models.PayloadField(key="labels")
            )
        ]
    )
)
print(len(scroll_result[0]))

for point in scroll_result[0]:
    if len(point.payload["xai"]) != 1:
        print("huh", point.payload["id"])

    shap_values = point.payload["xai"][0]["xai_values"]["token_scores"]
    # check if each value is in the range [-1, 1]
    # the shap_values are of size [len(tokens), 17]
    for token_scores in shap_values:
        for score in token_scores:
            if score < -0.1 or score > 0.1:
                print(point.payload["id"])
                break

print()
res = scroll_result[0][6]

print(res.payload["id"])
print(res.payload["labels"])
print(res.payload["xai"][0]["xai_method"])
print(res.payload["xai"][0]["predicted_label"])
print(res.payload["xai"][0]["probs"])
print(res.payload["xai"][1]["xai_method"])
print(res.payload["xai"][1]["predicted_label"])
print(res.payload["xai"][1]["probs"])
print()
print(res.payload["xai"][0]["xai_values"]["input_tokens"])
print(res.payload["xai"][1]["xai_values"]["input_tokens"])
#print(res.payload["xai"][2]["xai_values"]["input_tokens"])
print()
print(res.payload["xai"][0]["xai_values"]["token_scores"])
print(res.payload["xai"][1]["xai_values"]["token_scores"])
#print(res.payload["xai"][2]["xai_values"]["token_scores"])
print(res.payload["description"])

384
huh oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:www.zora.uzh.ch:62390
oai:

In [60]:
# check if all 384 points contain all xai methods and only them
xai_methods = set(["shap-partition", "shap-partition-tfidf", "attnlrp", "cplrp"])
for point in scroll_result[0]:
    xai_methods_for_this_point = xai_methods.copy()
    for xai in point.payload["xai"]:
        assert xai["xai_method"] in xai_methods_for_this_point, f"Publication {point.payload['id']} has unknown xai method '{xai['xai_method']}'"
        xai_methods_for_this_point.remove(xai["xai_method"])

    assert len(xai_methods_for_this_point) == 0, f"Publication {point.payload['id']} does not contain these expected xai methods: {xai_methods_for_this_point}"

In [61]:
# compare the sizes of token_scores for each xai method
for obj in scroll_result[0][0].payload["xai"]:
    print("XAI method:", obj["xai_method"])
    print(len(obj["xai_values"]["token_scores"]))
    print(obj["xai_values"]["input_tokens"])
    print("probs", obj["probs"])
    assert len(obj["xai_values"]["token_scores"]) == len(obj["xai_values"]["input_tokens"])

    for token_scores in obj["xai_values"]["token_scores"]:
        assert len(token_scores) == 17
        # assert that each token score is in the range [-1, 1]
        for score in token_scores:
            if score is None:
                raise ValueError(f"Publication {scroll_result[0][0].payload['id']} has a token score of None")
            assert -1 <= score <= 1, f"Publication {scroll_result[0][0].payload['id']} has a token score outside the range [-1, 1]: {score}"

attnlrp
142
['', 'Rein', 'sur', 'ance ', 'or ', 'Sec', 'uri', 'tization', ': ', 'The ', 'Case ', 'of ', 'Natural ', 'Cat', 'astro', 'ph', 'e ', 'Risk ', 'We ', 'investigate ', 'the ', 'suitability ', 'of ', 'sec', 'uri', 'tization ', 'as ', 'an ', 'alternative ', 'to ', 'rein', 'sur', 'ance ', 'for ', 'the ', 'purpose ', 'of ', 'transferring ', 'natural ', 'catast', 'roph', 'e ', 'risk', '. ', 'We ', 'characterize ', 'the ', 'conditions ', 'under ', 'which ', 'one ', 'or ', 'the ', 'other ', 'form ', 'of ', 'risk ', 'transfer ', 'dominates ', 'using ', 'a ', 'setting ', 'in ', 'which ', 'rein', 'sure', 'rs ', 'and ', 'trade', 'rs ', 'in ', 'financial ', 'markets ', 'produce ', 'costly ', 'information ', 'about ', 'catast', 'roph', 'es', '. ', 'Such ', 'information ', 'is ', 'useful ', 'to ', 'insure', 'rs', ': ', 'along ', 'with ', 'the ', 'information ', 'produced ', 'by ', 'insure', 'rs ', 'themselves', ', ', 'it ', 'reduces ', 'insure', 'rs ', '’ ', 'costly ', 'capital ', 'requireme

In [51]:
# # # for each point use client.set_payload to remove shap-partition-tfidf from the xai methods
# for point in scroll_result[0]:
#     new_xai = []
#     for xai in point.payload["xai"]:
#         if xai["xai_method"] != "shap-partition":
#             new_xai.append(xai)

#     client.set_payload(
#         collection_name="publications",
#         points=[point.id],
#         payload={"xai": new_xai}
#     )

In [57]:
# # for each point use client.set_payload to wrap the shap-partition-tfidf probabilities in a list
# for point in scroll_result[0]:
#     new_xai = []
#     for xai in point.payload["xai"]:
#         if xai["xai_method"] == "shap-partition":
#             xai["probs"] = [xai["probs"]]
#         new_xai.append(xai)

#     client.set_payload(
#         collection_name="publications",
#         points=[point.id],
#         payload={"xai": new_xai}
#     )

In [17]:
# compare the sizes of token_scores for each xai method
for obj in scroll_result[0][0].payload["xai"]:
    print(obj["xai_method"])
    print(len(obj["xai_values"]["token_scores"]))
    print(obj["xai_values"]["input_tokens"])
    print(len(obj["xai_values"]["token_scores"][0]))
    print(obj["xai_values"]["token_scores"][0])

    for token_scores in obj["xai_values"]["token_scores"]:
        assert len(token_scores) == 17
        # assert that each token score is in the range [-1, 1]
        for score in token_scores:
            assert -1 <= score <= 1

shap-partition
141
['', 'Rein', 'sur', 'ance ', 'or ', 'Sec', 'uri', 'tization', ': ', 'The ', 'Case ', 'of ', 'Natural ', 'Catast', 'roph', 'e ', 'Risk ', 'We ', 'investigate ', 'the ', 'suitability ', 'of ', 'sec', 'uri', 'tization ', 'as ', 'an ', 'alternative ', 'to ', 'rein', 'sur', 'ance ', 'for ', 'the ', 'purpose ', 'of ', 'transferring ', 'natural ', 'catast', 'roph', 'e ', 'risk', '. ', 'We ', 'characterize ', 'the ', 'conditions ', 'under ', 'which ', 'one ', 'or ', 'the ', 'other ', 'form ', 'of ', 'risk ', 'transfer ', 'dominates ', 'using ', 'a ', 'setting ', 'in ', 'which ', 'rein', 'sure', 'rs ', 'and ', 'trade', 'rs ', 'in ', 'financial ', 'markets ', 'produce ', 'costly ', 'information ', 'about ', 'catast', 'roph', 'es', '. ', 'Such ', 'information ', 'is ', 'useful ', 'to ', 'insure', 'rs', ': ', 'along ', 'with ', 'the ', 'information ', 'produced ', 'by ', 'insure', 'rs ', 'themselves', ', ', 'it ', 'reduces ', 'insure', 'rs', '’ ', 'costly ', 'capital ', 'require

AssertionError: 

In [19]:
xai_out_dict = []

for point in scroll_result[0]:
    xai_out_dict.append(
        {
            "id": point.payload["id"],
            "xai": point.payload["xai"],
        }
    )

In [ ]:
# save xai_out_dict on disk as json
# import json

# with open("xai_out_dict.json", "w") as f:
#    json.dump(xai_out_dict, f)


In [ ]:
points_with_labels_count = client.count(
    collection_name="publications",
    count_filter=models.Filter(
        must_not=[
            models.IsNullCondition(
                is_null=models.PayloadField(key="labels")
            )
        ]
    ),
    exact=True
)
print(points_with_labels_count)

sdgs = [f"sdg{i}" for i in range(1, 18)]
sdg_counts = {}

for sdg in sdgs:
    result = client.count(
        collection_name="publications",
        count_filter=models.Filter(
            must_not=[
                models.IsEmptyCondition(
                    is_empty=models.PayloadField(key=f"labels.{sdg}")
                )
            ]
        ),
        exact=True
    )
    sdg_counts[sdg] = result.count

print(sdg_counts)
print(sum(sdg_counts.values()))

count=384
{'sdg1': 6, 'sdg2': 13, 'sdg3': 43, 'sdg4': 2, 'sdg5': 22, 'sdg6': 3, 'sdg7': 17, 'sdg8': 26, 'sdg9': 13, 'sdg10': 31, 'sdg11': 3, 'sdg12': 30, 'sdg13': 41, 'sdg14': 14, 'sdg15': 79, 'sdg16': 34, 'sdg17': 7}
384


In [ ]:
assert sum(sdg_counts.values()) == points_with_labels_count.count

In [ ]:
client.count(
    collection_name="publications",
    count_filter=models.Filter(
        must_not=[
            models.IsEmptyCondition(
                is_empty=models.PayloadField(key=f"xai")
            )
        ]
    ),
    exact=True
)

CountResult(count=384)